<a href="https://colab.research.google.com/github/thekhan314/CovidBrazilMod3/blob/master/COVID_REDUX_googlepost.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Subset Features into floats and categoricals

## Libraries and Functions


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
import sys
sys.path.append('/content/drive/My Drive/Colab Notebooks/Mod3ProjectCOvid/CovidBrazilMod3')

from khantools import *
import pandas as pd
import numpy as np
import missingno
import pprint as pp
import matplotlib.pyplot as plt
import operator as operator

from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import roc_curve, auc
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import GridSearchCV

from sklearn.linear_model import LinearRegression, LogisticRegression, Lasso, Ridge
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', -1)

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm
/usr/local/lib/python3.6/dist-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.6/dist-packages/

In [0]:
# Evaluate empty columns
def cols_with_data(dataframe,threshold,plot=False,axis=0):
    ''' returns a series with all columns that are filled to at least threshold percentage or higher'''
    
    counts = dataframe.count(axis=axis).sort_values()
    
    if axis == 0: 
        counts = counts/len(dataframe)
    else:
        counts = counts/len(dataframe.columns)
        
    non_zero_cols = counts[counts >= threshold ]
    
    if plot == True:
        fig,ax = plt.subplots()
        ax.barh(non_zero_cols.index,non_zero_cols)
        fig.set_size_inches(6,9)
        fig.show()
        print("Features left:{}".format(len(non_zero_cols)))
    return non_zero_cols

def value_counts (df,value, oper):
    ''' takes in a datframe of float values, returns count of zero values '''
    dict = {}
    
    ops = {'>': operator.gt,
           '<': operator.lt,
           '>=': operator.ge,
           '<=': operator.le,
           '=': operator.eq}
    
    for col in df.columns:
        matches = sum(ops[oper](df[col],value))
        
        dict[col] = matches
    
    df = pd.Series(dict)
    return df


Lets import the data and run a preliminary report

In [0]:
df = pd.read_excel('/content/drive/My Drive/Colab Notebooks/Mod3ProjectCOvid/CovidBrazilMod3/dataset.xlsx')
report_prelim = report1(df,5)

It appears that the most commonly occuring values aside from the ones that are always there is what appears to be a battery of standard tests. I think it might be a good idea to discard all the rows where these tests were not performed, as they are much too sparse to be of any use to us. Lets keep only rows where standard tests were done, and see if that leaves us with enough data to make a meaningful model.

In [5]:
df = df[df['Parainfluenza 2'].notnull()]
report_post_trim = report1(df,6)
print(len(df))
#display(report_post_trim)

1352


# Data Cleanup and sorting

## Rename & Clean Columns

In [0]:
# Lets change the names of some columns to something easier to work with
new_names = {
    'Patient addmited to intensive care unit (1=yes, 0=no)':'intensive',
    'Patient addmited to semi-intensive unit (1=yes, 0=no)':'semi_int',
    'Patient addmited to regular ward (1=yes, 0=no)':'reg_ward',
    'SARS-Cov-2 exam result':'sars_cov2',
    'Patient age quantile':'age_quant',
    'Patient ID':'patient_id'
}

df = df.rename(columns=new_names)
df.set_index(keys='patient_id',inplace=True)

# convert age_quant to float

df['age_quant'] = df['age_quant'].astype(float)
#delete empty columns

empty_cols = list(report_post_trim[report_post_trim['% empty'] == 100.0].index)
df = df.drop(columns=empty_cols)



## Reduce features

In [0]:
# all data: filled values in each column 
col_fill = cols_with_data(df,threshold=0.03)

meaty_cols = list(col_fill[col_fill > 0.1].index)
meaty_cols.append('Lactic Dehydrogenase')

df = df[meaty_cols]

## Subset features

In [0]:
df_types = df.dtypes

float_features = list(df_types[df_types == 'float64'].index)
obj_features = list(df_types[df_types == 'object'].index)

high_imp= ['Lactic Dehydrogenase','Lymphocytes','Proteina C reativa mg/dL']
admission_features = ['intensive','semi_int','reg_ward']



for col in admission_features:
    df[col] = df[col].astype(float)

## Convert Categoricals

In [9]:
report2 = report1(df[obj_features],3).sort_values('unique values')
display(report2)

,type,% empty,unique values,1nth_value,1nth_value_%,2nth_value,2nth_value_%
Parainfluenza 2,object,0.00,1,not_detected,100.00,NaN,NaN
"Influenza B, rapid test",object,80.40,2,negative,18.49,positive,1.11
Rhinovirus/Enterovirus,object,0.00,2,not_detected,71.97,detected,28.03
sars_cov2,object,0.00,2,negative,91.72,positive,8.28
Inf A H1N1 2009,object,0.00,2,not_detected,92.75,detected,7.25
Influenza B,object,0.00,2,not_detected,94.38,detected,5.62
Respiratory Syncytial Virus,object,0.00,2,not_detected,96.15,detected,3.85
CoronavirusNL63,object,0.00,2,not_detected,96.67,detected,3.33
Coronavirus HKU1,object,0.00,2,not_detected,98.52,detected,1.48
"Influenza A, rapid test",object,80.40,2,negative,18.20,positive,1.41


Lets also drop all the unary features because it seems they are all basically negative and dont seem to add much information.

Lets convert the binary categoricals to floats. 

For tests that have a 'not done' value, I will treat 'not done' as negative, since its safe to assume if t he test wasnt done the doctor was expecting a negative anyways. 

Urine Aspect color and crystals I will drop. They dont sound too important, and dont comprise alot of rows.I will also drop standard tests, as I feel they will only add noise.  

Urine PH and Urine Leukocytes should be converted to float



In [10]:
binary_conversions = list(report2[report2['unique values'] == 2].index)

for col in binary_conversions:
    df[col] = df[col].replace(['not_detected','negative','absent','normal'],0)
    df[col] = df[col].replace(['detected','positive','not_done'],1)
    df[col] = df[col].astype(float)

df['sars_cov2'] = df['sars_cov2'].replace(['negative'],0)
df['sars_cov2'] = df['sars_cov2'].replace(['positive'],1)
df.drop('Parainfluenza 2',axis=1,inplace=True)

report3 = report1(df,6)

display(report3)

,type,% empty,unique values,1nth_value,1nth_value_%,2nth_value,2nth_value_%,3nth_value,3nth_value_%,4nth_value,4nth_value_%,5nth_value,5nth_value_%,6nth_value,6nth_value_%
Bordetella pertussis,float64,0.00,2,0.000000,99.85,1.000000,0.15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Parainfluenza 1,float64,0.00,2,0.000000,99.78,1.000000,0.22,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
CoronavirusOC43,float64,0.00,2,0.000000,99.41,1.000000,0.59,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Coronavirus229E,float64,0.00,2,0.000000,99.33,1.000000,0.67,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Chlamydophila pneumoniae,float64,0.00,2,0.000000,99.33,1.000000,0.67,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Parainfluenza 3,float64,0.00,2,0.000000,99.26,1.000000,0.74,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Adenovirus,float64,0.00,2,0.000000,99.04,1.000000,0.96,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Metapneumovirus,float64,0.00,2,0.000000,98.96,1.000000,1.04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Influenza A,float64,0.00,2,0.000000,98.67,1.000000,1.33,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Parainfluenza 4,float64,0.00,2,0.000000,98.59,1.000000,1.41,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Fill Empties

In [0]:
full_cols = cols_with_data(df,1)
full_feats = list(full_cols.index)

unfull_feats = list(df.drop(full_feats,axis=1).columns)


In [0]:
filled_feats = full_feats.copy()


for feature in unfull_feats:
    df_vals_present = df.loc[df[feature].notnull()]
    
    X = df_vals_present[filled_feats]
    y = df_vals_present[feature]
    
    scaler = StandardScaler()
    X = scaler.fit_transform(X)
    
    if feature in float_features:
        pipe = Ridge(alpha=0.5)
       
        
    elif feature in obj_features:
        
        pipe = LogisticRegression(C=1e5, solver='lbfgs',class_weight='balanced',max_iter=10000)
        
    
    pipe.fit(X,y)
    
    df_predictors = df.loc[df[feature].isna()][filled_feats]
    y_hat = pipe.predict(df_predictors)
    
    df.loc[df[feature].isna(),feature] = y_hat
    
    filled_feats.append(feature)

In [13]:
display(report1(df,4))

,type,% empty,unique values,1nth_value,1nth_value_%,2nth_value,2nth_value_%,3nth_value,3nth_value_%,4nth_value,4nth_value_%
Bordetella pertussis,float64,0.0,2,0.000000,99.85,1.000000,0.15,NaN,NaN,NaN,NaN
Parainfluenza 1,float64,0.0,2,0.000000,99.78,1.000000,0.22,NaN,NaN,NaN,NaN
CoronavirusOC43,float64,0.0,2,0.000000,99.41,1.000000,0.59,NaN,NaN,NaN,NaN
Chlamydophila pneumoniae,float64,0.0,2,0.000000,99.33,1.000000,0.67,NaN,NaN,NaN,NaN
Coronavirus229E,float64,0.0,2,0.000000,99.33,1.000000,0.67,NaN,NaN,NaN,NaN
Parainfluenza 3,float64,0.0,2,0.000000,99.26,1.000000,0.74,NaN,NaN,NaN,NaN
Adenovirus,float64,0.0,2,0.000000,99.04,1.000000,0.96,NaN,NaN,NaN,NaN
Metapneumovirus,float64,0.0,2,0.000000,98.96,1.000000,1.04,NaN,NaN,NaN,NaN
Influenza A,float64,0.0,2,0.000000,98.67,1.000000,1.33,NaN,NaN,NaN,NaN
"Influenza A, rapid test",float64,0.0,2,0.000000,98.59,1.000000,1.41,NaN,NaN,NaN,NaN


## Unified target column

In [0]:
for feat in admission_features:
    df.loc[df[feat] == 1,'adm_status'] = feat

df['adm_status'].fillna(value='not_adm',inplace=True)
df = df.drop(columns=admission_features)

# Train Classifiers

In [0]:
from imblearn.over_sampling import SMOTE


In [16]:
smote = SMOTE()

X = df.drop('adm_status',axis=1)
scaler = StandardScaler()
X = scaler.fit_transform(X)
y = df['adm_status']

X_train, X_test, y_train, y_test = train_test_split(X,y,  test_size=0.25, random_state=42)
X_train, y_train = smote.fit_sample(X_train, y_train) 

/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


In [17]:
print(pd.Series(y_train).value_counts()) 

intensive    915
not_adm      915
semi_int     915
reg_ward     915
dtype: int64


## Logistic Regression

In [0]:
weights_dict = [{
    'not_adm':1,
    'reg_ward':10,
    'semi_int':100,
    'intensive':200
},{
    'not_adm':1,
    'reg_ward':25,
    'semi_int':200,
    'intensive':500
},{
    'not_adm':1,
    'reg_ward':50,
    'semi_int':400,
    'intensive':750
},{
    'not_adm':1,
    'reg_ward':100,
    'semi_int':300,
    'intensive':1000
}
]

In [0]:
clf = LogisticRegression()

params = {
    'C': [1e2,1e5,1e10,1e20],
    'solver':['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga'],
    'max_iter':[100,1000,10000],
    'class_weight':weights_dict
}

gs_logistic = GridSearchCV(estimator=clf,param_grid=params,cv=5)
gs_logistic.fit(X_train,y_train)


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression


In [19]:


clf = LogisticRegression(C=1e5, solver='lbfgs',multi_class='auto',max_iter=100000)
clf.fit(X_train,y_train)




y_score = clf.predict(X_test)

class_report_1 = classification_report(y_test,y_score)

print(class_report_1)

              precision    recall  f1-score   support

   intensive       0.75      0.60      0.67        10
     not_adm       0.98      0.92      0.95       305
    reg_ward       0.33      0.50      0.40        16
    semi_int       0.20      0.57      0.30         7

    accuracy                           0.88       338
   macro avg       0.57      0.65      0.58       338
weighted avg       0.93      0.88      0.90       338



## Decision Tree

In [0]:
clf = DecisionTreeClassifier()

params = {
    'criterion': ['entropy','gini'],
    'splitter':['best', 'random'],
    'max_depth':[i for i in np.linspace(1, 32, 32, endpoint=True)],
    'min_samples_split':[i for i in np.linspace(0.1, 1.0, 10, endpoint=True)],
    'min_samples_leaf':[i for i in np.linspace(0.1, 0.5, 5, endpoint=True)]
}

gs_dec_tree = GridSearchCV(estimator=clf,param_grid=params,cv=5)
gs_dec_tree.fit(X_train,y_train)

gs_dec_tree.best_score_
gs_dec_tree.best_params_

## Random Forest

In [0]:
clf = DecisionTreeClassifier()

params = {
    'criterion': ['entropy','gini'],
    'splitter':['best', 'random'],
    'max_depth':[i for i in np.linspace(1, 32, 32, endpoint=True)],
    'min_samples_split':[i for i in np.linspace(0.1, 1.0, 10, endpoint=True)],
    'min_samples_leaf':[i for i in np.linspace(0.1, 0.5, 5, endpoint=True)]
}

gs_dec_tree = GridSearchCV(estimator=clf,param_grid=params,cv=5)
gs_dec_tree.fit(X_train,y_train)

gs_dec_tree.best_score_
gs_dec_tree.best_params_

In [20]:


clf = RandomForestClassifier()
clf.fit(X_train,y_train)
 
y_score = clf.predict(X_test)

class_report_1 = classification_report(y_test,y_score)

print(class_report_1)

              precision    recall  f1-score   support

   intensive       0.67      0.40      0.50        10
     not_adm       0.98      0.96      0.97       305
    reg_ward       0.41      0.56      0.47        16
    semi_int       0.08      0.14      0.11         7

    accuracy                           0.91       338
   macro avg       0.54      0.52      0.51       338
weighted avg       0.93      0.91      0.92       338



## XGboost

In [21]:
import xgboost as xgb
clf = xgb.XGBClassifier( objective='multi:softmax', colsample_bytree = 0.3, learning_rate = 0.1,
                max_depth = 5, alpha = 10, n_estimators = 10,scale_pos_weight=15)
clf.fit(X_train,y_train)

y_score = clf.predict(X_test)

class_report_1 = classification_report(y_test,y_score)

print(class_report_1)

              precision    recall  f1-score   support

   intensive       0.46      0.60      0.52        10
     not_adm       0.99      0.93      0.96       305
    reg_ward       0.26      0.38      0.31        16
    semi_int       0.00      0.00      0.00         7

    accuracy                           0.88       338
   macro avg       0.43      0.48      0.45       338
weighted avg       0.92      0.88      0.89       338



## Support Vector Machines